In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

import sqlite3
import re

# Парсинг сайта и настройка базы данных

## 1. Парсинг

In [2]:
# функция очистки текста от лишних символов
def replace_all(text, to_replace):
    for symbol in to_replace:
        text = text.replace(symbol, ' ')
    return text

In [3]:
# ссылки на исполнителей
urls = ['https://txtsong.ru/krovostok/', 'https://txtsong.ru/xaski/', 'https://txtsong.ru/elektroforez/', 'https://txtsong.ru/molchat-doma/', 'https://txtsong.ru/durnoj-vkus/', 'https://txtsong.ru/akvarium/',
       'https://txtsong.ru/skriptonit/', 'https://txtsong.ru/splin/', 'https://txtsong.ru/zemfira/', 'https://txtsong.ru/smoki-mo/', 'https://txtsong.ru/maslo-chernogo-tmina/', 'https://txtsong.ru/2h-company/',
        'https://txtsong.ru/buerak/', 'https://txtsong.ru/agata-kristi/', 'https://txtsong.ru/akvarium/', 'https://txtsong.ru/srub/', 'https://txtsong.ru/aleksandr-f-sklyar/', 'https://txtsong.ru/yanka-dyagileva/',
       'https://txtsong.ru/green-apelsin/', 'https://txtsong.ru/korol-i-shut/', 'https://txtsong.ru/ic3peak/', 'https://txtsong.ru/grechka/']

# создадим список с ссылками на тексты
songs_lists = []

for url in urls:
    songs_list = requests.get(url)
    songs_soup = BeautifulSoup(songs_list.content, 'html.parser')

    for line in songs_soup.find_all('a'):  # вытащим из html адреса с текстами и добавим их в список
        if str(line).find(url[19:]) != -1 and str(line).find('/page/') == -1:
            song_url = line['href']
            songs_lists.append(song_url)

songs_lists = list(set(songs_lists)) # удалим дубли
print(len(songs_lists))

1637


In [4]:
songs_lists[0:5]

['https://txtsong.ru/grechka/grechka-vy-zhalkie/',
 'https://txtsong.ru/skriptonit/rajda-skriptonit-truwer-zhenshhiny/',
 'https://txtsong.ru/splin/splin-moya-lyubov/',
 'https://txtsong.ru/korol-i-shut/korol-i-shut-mest-garri/',
 'https://txtsong.ru/skriptonit/niman-skriptonit-kali-4-20/']

In [8]:
# список с символами для удаления из текстов
symbols_to_del = ['!', '?', '.', ',', ':', '-', '"', '—', '  ', '–', '“', '“', '<p>', '<br/>', '</br>', '<br>', '</p>', '\n', '(', ')', '«', '»', '\u2005', '\u205f', '\xa0', '</em>', '<em>', '\\', 'u200', '<ul>', '<li>', '</li>', '</ul>', "<  Yandex RTB R A 1318229 42  > <div id= yandex_rtb_R A 1318229 42 ></div> <script>window yaContextCb push   =>{  Ya Context AdvManager render {   renderTo 'yandex_rtb_R A 1318229 42'    blockId 'R A 1318229 42'  }  } </script>"]

In [9]:
parsed_songs = {}

for song in songs_lists:
    song_url = requests.get(song)
    song_soup = BeautifulSoup(song_url.content, 'html.parser')
    song_text = str(song_soup.find_all('p')[4]) # находим нужную часть с текстом
    song_text = replace_all(song_text, symbols_to_del)  # очищаем тексты от лишних символов  
    song_text = song_text.rstrip().lstrip() # убираем лишние пробелы в начале и в конце
    parsed_songs[song] = song_text
    time.sleep(1) # создает интервал между действиями, чтобы нас не забанили

In [10]:
len(parsed_songs)

1637

In [13]:
#parsed_songs

## 2. Работа с БД

In [6]:
con = sqlite3.connect('songs.db') # создаем локальную базу данных

In [15]:
cur = con.cursor()
cur.execute('CREATE TABLE song_texts2(author, text, url, date)') # создаем таблицу
res = cur.execute('SELECT name FROM sqlite_master')
res.fetchone() # проерим создалась ли таблица

for k, v in parsed_songs.items():   # перебираем ключи и значения и записываем их в таблицу

    sql_insert = """
                    INSERT INTO song_texts2 (author, text, url, date)
                    VALUES ('{}', '{}', '{}', '{}')
                    """.format(k.split('/')[3], v, k, datetime.today())

    res = cur.execute(sql_insert)
    con.commit()
    
res = cur.execute('SELECT * FROM song_texts2')
result = res.fetchall()

In [16]:
result[0]

('grechka',
 'Слухи сплетни город мал   Ты просто сидишь никого не обидев;  Но каждый в округе знает что ты   Дура тупая и шкура простая;  Ты дура тупая и шкура простая      Вот так в пяти словах о тебе   Ты даже не дышишь не ешь и не пьёшь   Для этих людей по словам от других   Дура тупая и шкура простая;  Ты дура тупая и шкура простая      И пусть ты просто парня отшила;  И пусть ты просто лучше других   Неважно народу в этой округе   Ведь ты дура тупая и шкура простая   Ты дура тупая и шкура простая    Вот так в пяти словах о тебе   Ты даже не дышишь не ешь и не пьешь   Для этих людей по словам от других      Ты дура тупая и шкура простая;  Ты дура тупая и шкура простая;  Ты дура тупая и шкура простая;  Ты дура тупая и шкура простая      Если вдруг кто то меня недопонял   То песня об этих уродах тупых   Что сплетни о других несут в народы   Вы жалкие крысы съешьте свой пыл',
 'https://txtsong.ru/grechka/grechka-vy-zhalkie/',
 '2023-01-31 20:45:03.066345')

In [17]:
# подготовим тексты 
text_prep_all = []
artist_name = []
for i in range(len(result)):
    text_prep = re.findall('[А-ЯA-Z][^А-ЯA-Z]*', result[i][1]) # делим текст на строки
    text_prep = [a.lower() for a in text_prep] # убираем заглавные буквы
    text_prep = [a.rstrip().rstrip() for a in text_prep] # удаляем лишние пробелы
    text_prep = [a.lstrip() for a in text_prep]
    
    clean_text = [] # убираем короткие и пустые строки
    for val in text_prep:
        if len(val) <= 10:
            pass
        else:
            clean_text.append(val)
    
    if len(clean_text) < 4: # убираем короткие тексты
        pass
    else:
        text_prep_unique = list(set(clean_text)) # оставляем уникальные строки в текстах
        text_prep_str = '\n'.join(text_prep_unique) # добавляем символ табуляции
        text_prep_all.append(text_prep_str) # итоговый список с текстами
        artist_name.append(result[i][0]) # список с именами исполнителей

In [18]:
# создадим датафрейм и сохраним в csv
df_prepped = pd.DataFrame()


df_prepped['text'] = text_prep_all * 3
df_prepped['artist'] = artist_name * 3
df_prepped = df_prepped[['artist', 'text']]
df_prepped.to_csv('~/GIT/Projects/ML-сервис/lyrics-generator/dif_texts_6.csv', index=False)


print(df_prepped.shape)
df_prepped.head()

(4860, 2)


,artist,text
0,grechka,то песня об этих уродах тупых\nвы жалкие крысы...
1,skriptonit,нас не найти к черту условности\nсеверном полю...
2,splin,и никомy не довеpяй ключи от дома не клянись н...
3,korol-i-shut,гарри нам удружил в еду мерзавец яду подложил...
4,skriptonit,слово в цене как и раньше превыше всех цацек\n...
